In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [4]:
geneesmiddelen = pd.read_csv('..\Datasets\metadata.csv', sep='|')
hulpstoffen = pd.read_csv('..\Datasets\keto_hulpstoffen_GIB_uniek_20220712.csv', sep='\t', encoding='latin')

In [5]:
yes = hulpstoffen[hulpstoffen.Keto == 'J']['hulpstof'].to_list()
possible = hulpstoffen[hulpstoffen.Keto == 'M']['hulpstof'].to_list()
no = hulpstoffen[hulpstoffen.Keto == 'N']['hulpstof'].to_list()
unknown = hulpstoffen[hulpstoffen.Keto == '?']['hulpstof'].to_list()

In [6]:
'''Volgorde labeling: N > M > ? > J. Anders wordt er onterecht overgeschreven'''

def prep(value):
    out=[]
    for i in str(value).split('#'):
        str(i).strip()
        out.append(i)
    return out

## LABELEN ##

try:
    #geneesmiddelen['gluten'] = geneesmiddelen['HULPSTOFFEN'].apply(lambda x: 'J' if any(str(hulpstof) in str(x) for hulpstof in yes) else ('M' if any(str(hulpstof) in str(x) for hulpstof in possible) else ('N' if any(str(hulpstof) in str(x) for hulpstof in no) else None)))
    geneesmiddelen['keto'] = geneesmiddelen['HULPSTOFFEN'].apply(lambda x: 'N' if any(str(hulpstof) in prep(x) for hulpstof in no) 
    else ('M' if any(str(hulpstof) in prep(x) for hulpstof in possible) 
    else ('?' if any(str(hulpstof) in prep(x) for hulpstof in unknown) 
    else ('J' if any(str(hulpstof) in prep(x) for hulpstof in yes) else None))))

except Exception as err:
    print(err)


In [8]:
today = dt.datetime.today().strftime('%Y%m%d')
name = '..\Datasets\geneesmiddelen_keto_labeled_%s.csv'  %(today)

geneesmiddelen.to_csv(name, sep=';', encoding='utf-8' )

In [9]:
import pandas as pd
from sqlalchemy import create_engine

# Credentials to database connection
hostname="185.224.137.5"
dbname="u219831890_ketomed"
uname="u219831890_ketobeheer"
pwd="Admin_admin1"

# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

# Convert dataframe to sql table  
sql_db_name = 'cbg_labeled_%s' %(today)                                 
geneesmiddelen.to_sql(sql_db_name, engine, index=False)


19276